<a href="https://colab.research.google.com/github/Ngadella/Image-sim-comp/blob/main/Model2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install keras==3.3.3

In [ ]:
pip install tensorflow==2.16.1

In [ ]:
pip show tensorflow keras

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(64, 64, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
embeddings = Dense(128, activation='relu')(x)
model = Model(inputs=base_model.input, outputs=embeddings)

model.compile(optimizer='adam', loss='categorical_crossentropy')

In [ ]:
import cv2
img1_raw = cv2.imread('/content/0.jpg')
img2_raw = cv2.imread('/content/2.jpg')

print(img1_raw.shape)
print(img2_raw.shape)

img1 = cv2.resize(img1_raw, (64, 64))
img2 = cv2.resize(img2_raw, (64, 64))

print(img1.shape)
print(img2.shape)

img1 = img1.reshape((1, 64, 64, 3))
img2 = img2.reshape((1, 64, 64, 3))

emb1 = model.predict(img1)
emb2 = model.predict(img2)

In [ ]:
print(emb1.shape)
print(emb2.shape)

In [ ]:

import numpy as np
from scipy import spatial

cosine_similarity = 1 - spatial.distance.cosine(emb1[0], emb2[0])
print(cosine_similarity)


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('/content/model.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
!xxd -i /content/model.tflite > model.h